Importing lib

In [2]:
from pyspark import SparkConf, SparkContext
from pyspark.ml.linalg import Vectors
import random
import time
import numpy as np

In [3]:

config = SparkConf().setAppName('Homework 3').setMaster('local')

sc = SparkContext(conf=config)

Useful functions from the past

In [4]:
#K-Center fast implementation with Fartherst-Traversal First
def kcenter(P, k):
    #P_minus_S = [p for p in P]
    idx_rnd = random.randint(0, len(P)-1)
    S = [P[idx_rnd]]
    dist_near_center = [Vectors.squared_distance(P[i], S[0]) for i in range(len(P))]

    for i in range(k-1):    
        new_center_idx = max(enumerate(dist_near_center), key=lambda x: x[1])[0] # argmax operation
        S.append(P[new_center_idx])

        for j in range(len(P)):
            if j != new_center_idx:
                dist = Vectors.squared_distance(P[j], S[-1])
                if dist < dist_near_center[j]:
                    dist_near_center[j] = dist
            else:
                dist_near_center[j] = 0
    return S


#Import toy dataset RDD

def readVectorsSeq(filename):
    file = open(filename, 'r')
    vector_list = []
    for row in file.readlines():
        vector_list.append(Vectors.dense([float(num_str) for num_str in row.split()]))
    return vector_list

# Import the Dataset
vec_list = readVectorsSeq('test-datasets/prova.txt')

# Create a parallel collection
dNumbers = sc.parallelize(vec_list)



def runSequential(points, k):

    n = len(points)
    if k >= n:
        return points

    result = list()
    candidates = np.full(n , True)

    for iter in range(int(k / 2)):
        maxDist = 0.0
        maxI = 0
        maxJ = 0
        for i in range(n):
            if candidates[i] == True:
                for j in range(n):
                    d = Vectors.squared_distance(points[i], points[j])
                    if d > maxDist:
                        maxDist = d
                        maxI = i
                        maxJ = j
        result.append( points[maxI] )
        result.append( points[maxJ] )
        #print "selecting "+str(maxI)+" and "+str(maxJ)
        candidates[maxI] = False
        candidates[maxJ] = False

    if k % 2 != 0:
        s = np.random.randint(n)
        for i in range(n):
            j = (i + s) % n
            if candidates[j] == True:
                #print "selecting "+str(j)
                result.append( points[i] )
                break

    return result



In [5]:
# Import the Dataset and Define the Variables
datafile = 'test-datasets/prova.txt'

inputrdd = sc.textFile(datafile)\
                        .map(lambda row : Vectors.dense([float(num_str) for num_str in row.split(' ')]))

For this homework you need to develop the following two functions.

A function runMapReduce(pointsrdd,k,numBlocks) that receives in input a set of points represented by an RDD pointsrdd of Vector and two integers k and numBlocks, and does the following things: 

       (a) partitions pointsrdd into numBlocks subsets; 
   
       (b) extracts k points from each subset by running the sequential Farthest-First Traversal algorithm implemented for Homework 3; 
   
       (c) gathers the numBlocks*k points extracted into a list of Vector coreset; 
   
       (d) returns a a list of Vector with the k points determined by running the sequential max-diversity algorithm with input coreset and k. The code of the sequential algorithm can be downloaded here.
  

In [159]:
def getMatrix(x):
    mat = []
    for element in x:
        mat.append(element)
    return mat

def runMapReduce(pointsrdd, k, numBlocks):
    #partitions pointsrdd into numBlocks subsets
    distRDD = inputrdd.map(lambda x: (np.random.randint(numBlocks),x)).groupByKey() #map each vector with a random index. Given the uniform distribution, with a big dataset I expect the partition to be uniform.
    
    ##k-point extraction with Fartherst-First Traversal 
    coreset = distRDD.mapValues(lambda x: getMatrix(x)).mapValues(lambda x: 'AAAAAAAAAAA').collect()    
    
    # Extract k-Points maximal diversity on a single reducer
    #list_max_diversity_output = runSequential(coreset,k)
    
    return coreset

output = runMapReduce(inputrdd, 3, 3)


In [160]:
output = runMapReduce(inputrdd, 2, 3)
print(output)

[(1, 'AAAAAAAAAAA'), (0, 'AAAAAAAAAAA'), (2, 'AAAAAAAAAAA')]


 A function measure(pointslist) that receives in input a set of points represented as a list pointslist and returns a double which is the average distance between all points in pointslist (i.e., the sum of all pairwise distances divided by the number of distinct pairs). 

In [116]:
def measure(pointlist):
    cumsum = 0
    i=0
    for point_one in pointlist:
        for point_two in pointlist:
            if point_one != point_two:
                cumsum += np.linalg.norm(point_one-point_two, 2)
                i = i+1
    return cumsum/i

In [117]:
measure(output)

4.361685673643496

In [118]:
time.time()

1527518220.8152971

In [121]:
def runMapReduce(pointsrdd, k, numBlocks):
	before = time.time()
    #partitions pointsrdd into numBlocks subsets
	distRDD = inputrdd.map(lambda x: (np.random.randint(numBlocks),x)).groupByKey() #map each vector with a random index. Given the uniform distribution, with a big dataset I expect the partition to be uniform.
    
    ##k-point extraction with Fartherst-First Traversal 
    coreset = distRDD.mapValues(lambda x: getMatrix(x)).mapValues(lambda x: kcenter(x,k)).flatMap(lambda x: x[1]).collect()    
    print("Time taken for coreset construction: " + str(time.time()-before))
    # Extract k-Points maximal diversity on a single reducer

    before = time.time()
    list_max_diversity_output = runSequential(coreset,k)
    print("Time taken for completing sequential algorithm: " + str(time.time()-before))
   
    return list_max_diversity_output

IndentationError: unindent does not match any outer indentation level (<ipython-input-121-ad44d9e56c44>, line 7)

In [132]:
def runMapReduce(pointsrdd, k, numBlocks):
    #partitions pointsrdd into numBlocks subsets
    before = time.time()
    distRDD = inputrdd.map(lambda x: (np.random.randint(numBlocks),x)).groupByKey() #map each vector with a random index. Given the uniform distribution, with a big dataset I expect the partition to be uniform.
    
    ##k-point extraction with Fartherst-First Traversal 
    coreset = distRDD.mapValues(lambda x: getMatrix(x)).mapValues(lambda x: kcenter(x,k)).flatMap(lambda x: x[1]).collect()
    print("Time taken for coreset construction: " + str(time.time()-before))
    
    # Extract k-Points maximal diversity on a single reducer
    before = time.time()
    list_max_diversity_output = runSequential(coreset,k)
    print("Time taken for completing sequential algorithm: " + str(time.time()-before))

    return list_max_diversity_output

In [124]:
def runMapReduce(pointsrdd, k, numBlocks):
    
	before = time.time()
    #partitions pointsrdd into numBlocks subsets
    distRDD = inputrdd.map(lambda x: (np.random.randint(numBlocks),x)).groupByKey() #map each vector with a random index. Given the uniform distribution, with a big dataset I expect the partition to be uniform.
    
    ##k-point extraction with Fartherst-First Traversal 
    coreset = distRDD.mapValues(lambda x: getMatrix(x)).mapValues(lambda x: kcenter(x,k)).flatMap(lambda x: x[1]).collect()    
    print("Time taken for coreset construction: " + str(time.time()-before))
    
    # Extract k-Points maximal diversity on a single reducer
    before = time.time()
    list_max_diversity_output = runSequential(coreset,k)
    print("Time taken for completing sequential algorithm: " + str(time.time()-before))

    return list_max_diversity_output

IndentationError: unindent does not match any outer indentation level (<ipython-input-124-545f2bfcc635>, line 5)